In [14]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint

# .env 파일에서 환경 변수 로드
load_dotenv(dotenv_path='../.env')

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

print(client_id[:3], client_secret[:3])

GSJ n_N


In [15]:

def save_to_json(data, filepath):
    """
    데이터를 JSON 파일로 저장합니다. 디렉토리가 없으면 자동으로 생성합니다.

    Args:
        data (list or dict): 저장할 데이터.
        filepath (str): 저장할 파일 경로 (예: 'data/books.json').
    """
    try:
        # 파일 경로에서 디렉토리 부분만 추출하여 해당 디렉토리가 없으면 생성합니다.
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        # JSON 파일을 쓰기 모드('w')로 열고, 한글이 깨지지 않도록 인코딩을 'utf-8'로 설정합니다.
        with open(filepath, 'w', encoding='utf-8') as f:
            # json.dump를 사용하여 데이터를 파일에 씁니다.
            # ensure_ascii=False: 한글을 유니코드 이스케이프 시퀀스 대신 그대로 저장합니다.
            # indent=4: JSON 파일을 4칸 들여쓰기로 포맷하여 가독성을 높입니다.
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f" 데이터가 '{filepath}' 파일에 성공적으로 저장되었습니다.")

    except (IOError, TypeError) as e:
        print(f" 파일 저장 중 오류가 발생했습니다: {e}")


def search_books(query):
    """
    네이버 책 검색 API를 호출하여 검색 결과를 가져오고, JSON 파일로 저장합니다.

    Args:
        query (str): 검색할 키워드.

    Returns:
        list: 책 검색 결과 리스트. 오류 발생 시 빈 리스트를 반환합니다.
    """
    # 네이버 책 검색 API의 기본 URL
    url = 'https://openapi.naver.com/v1/search/book.json'

    # API 요청에 필요한 헤더 정보 (클라이언트 ID와 시크릿)
    headers = {
        'X-Naver-Client-Id': CLIENT_ID,
        'X-Naver-Client-Secret': CLIENT_SECRET
    }
    
    # API 요청에 필요한 파라미터 (검색어, 표시할 개수, 정렬 방식)
    payload = {
        'query': query,
        'display': 50,  # 결과를 최대 50개까지 표시
        'sort': 'sim'   # 유사도 순으로 정렬
    }

    try:
        # requests.get을 사용하여 API에 GET 요청을 보냅니다.
        res = requests.get(url, params=payload, headers=headers)
        # HTTP 상태 코드가 200 (OK)이 아닐 경우, 예외를 발생시킵니다.
        res.raise_for_status()

        # 응답 받은 JSON 데이터에서 'items' 키에 해당하는 책 목록을 추출합니다.
        items_data = res.json()['items']

        # 검색 결과가 있을 경우, 파일 저장 함수를 호출합니다.
        if items_data:
            save_to_json(items_data, 'data/books.json')
        else:
            print(" 검색 결과가 없습니다.")

        return items_data

    except requests.exceptions.RequestException as e:
        print(f" API 요청 중 오류가 발생했습니다: {e}")
        # 오류 발생 시, API 응답 내용을 확인하여 원인을 파악할 수 있습니다.
        # print("API 응답 내용:", res.text)
        return []
    except KeyError:
        print(" API 응답 형식에 'items' 키가 없습니다. 응답을 확인해주세요.")
        return []


# --- 메인 실행 블록 ---
if __name__ == "__main__":
    # 스크립트 실행 전, API 정보가 제대로 입력되었는지 확인합니다.
    if CLIENT_ID == 'YOUR_CLIENT_ID' or CLIENT_SECRET == 'YOUR_CLIENT_SECRET':
        print("  경고: 코드 상단의 CLIENT_ID와 CLIENT_SECRET을 실제 값으로 변경해주세요.")
    else:
        # 사용자로부터 검색어를 입력받습니다.
        search_query = input("검색할 책의 키워드를 입력하세요: ")
        
        if search_query:
            # search_books 함수를 호출하여 결과를 받아옵니다.
            book_list = search_books(search_query)
            if book_list:
                print(f"\n총 {len(book_list)}개의 검색 결과를 찾았습니다.")
        else:
            print("검색어를 입력해야 합니다.")

 API 요청 중 오류가 발생했습니다: 401 Client Error: Unauthorized for url: https://openapi.naver.com/v1/search/book.json?query=%EC%B1%84%EC%8B%9D%EC%A3%BC%EC%9D%98%EC%9E%90&display=50&sort=sim
